## Exercise 1

In [1]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from scipy.stats import expon, reciprocal
import tarfile
import urllib

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [3]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH)
housing=load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


##### Define Data Preparation Pipeline

In [5]:
# Separate features and target
X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"]

In [6]:
# Numerical and categorical columns
num_features = X.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X.select_dtypes(include=[object]).columns.tolist()

In [7]:
# Create a numerical transformer pipeline
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
    ('scaler', StandardScaler())
])

# Create a categorical transformer pipeline
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [8]:
# Combine into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)


In [9]:
# Define the full pipeline
prep_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(score_func=f_regression, k=10))
])

##### Train-Test Split

In [10]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Apply the pipeline to preprocess the training data
X_train_prepared = prep_pipeline.fit_transform(X_train, y_train)
X_test_prepared = prep_pipeline.transform(X_test)

## 1. Support Vector Regressor
Using GridSearchCV

In [12]:
# Define the parameter grid for GridSearchCV
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10]}
]
svr = SVR()

# Perform GridSearchCV
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train_prepared, y_train)

# Display the best parameters and model performance
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test_prepared)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ...............................C=0.1, kernel=linear; total time=  11.9s
[CV] END ...............................C=0.1, kernel=linear; total time=   9.4s
[CV] END ...............................C=0.1, kernel=linear; total time=   9.1s
[CV] END ...............................C=0.1, kernel=linear; total time=   8.4s
[CV] END ...............................C=0.1, kernel=linear; total time=   8.1s
[CV] END .................................C=1, kernel=linear; total time=   7.9s
[CV] END .................................C=1, kernel=linear; total time=   7.8s
[CV] END .................................C=1, kernel=linear; total time=   7.1s
[CV] END .................................C=1, kernel=linear; total time=   6.9s
[CV] END .................................C=1, kernel=linear; total time=   6.5s
[CV] END ................................C=10, kernel=linear; total time=   7.8s
[CV] END ................................C=10, 

### 2. Using RandomizedSearchCV:

In [13]:
# Define SVR pipeline
svr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svr', SVR())
])

# Correct parameter distribution with step prefix
param_distributions = {
    'svr__kernel': ['linear', 'rbf'],  # Note the 'svr__' prefix
    'svr__C': reciprocal(0.1, 100),
    'svr__gamma': expon(scale=1)  # Only applies if 'kernel' is 'rbf'
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svr_pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model
random_search.fit(X_train, y_train)

# Output the best parameters and RMSE
print("Best parameters:", random_search.best_params_)
print("Best RMSE:", np.sqrt(-random_search.best_score_))


Best parameters: {'svr__C': 89.0620438616168, 'svr__gamma': 0.628789100540856, 'svr__kernel': 'linear'}
Best RMSE: 72070.5042575946


### 3. Preparing pipeline to select only the most important attributes

In [14]:
# Assuming the following columns for numerical and categorical features
numerical_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
categorical_features = ['ocean_proximity']

# Preprocessing pipeline with feature selection
numerical_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('feature_selector', SelectKBest())  # Feature selection step
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

In [15]:
# Define parameter grid for grid search
svr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svr', SVR())
])

# Define parameter grid for grid search
param_grid_pipeline = {
    'preprocessor__num__feature_selector__k': [5], 
    'svr__C': [0.1, 1],
    'svr__gamma': [0.01]
}

# GridSearchCV
grid_search_pipeline = GridSearchCV(
    estimator=svr_pipeline,
    param_grid=param_grid_pipeline,
    cv=3,
    scoring='neg_mean_squared_error',
    error_score='raise'
)
grid_search_pipeline.fit(X_train, y_train)

print("Best parameters (Pipeline):", grid_search_pipeline.best_params_)
print("Best RMSE (Pipeline):", np.sqrt(-grid_search_pipeline.best_score_))

Best parameters (Pipeline): {'preprocessor__num__feature_selector__k': 5, 'svr__C': 1, 'svr__gamma': 0.01}
Best RMSE (Pipeline): 118699.21911830842


### 4. Creating a single pipeline that does the full data preparation plus the final prediction 

In [16]:
# Define the complete pipeline
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(score_func=f_regression, k=10)),  # Feature selection
    ('model', SVR())  # Final prediction model
])

### 5. Automatically explore some preparation options using GridSearchCV

In [17]:
# Assuming the following columns for numerical and categorical features
numerical_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
categorical_features = ['ocean_proximity']

# Preprocessing pipelines
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler()),
    ('feature_selector', SelectKBest(score_func=f_regression))  # Add score_func explicitly
])
categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)


# Full pipeline
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svr', SVR())
])

In [18]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'preprocessor__num__feature_selector__k': [5, 8],
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': [0.1, 1, 10, 100],
    'svr__gamma': ['scale', 'auto']
}

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# GridSearchCV
grid_search_pipeline = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    error_score='raise'
)

grid_search_pipeline.fit(X_train, y_train)

# Results
print("Best parameters:", grid_search_pipeline.best_params_)
print("Best RMSE:", np.sqrt(-grid_search_pipeline.best_score_))

Best parameters: {'preprocessor__num__feature_selector__k': 8, 'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'linear'}
Best RMSE: 71837.41930036288


In [19]:
X_train_prepared = prep_pipeline.fit_transform(X_train, y_train)

In [20]:
# Display the best parameters and performance
print("Best parameters:", grid_search_pipeline.best_params_)
best_model = grid_search_pipeline.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Best parameters: {'preprocessor__num__feature_selector__k': 8, 'svr__C': 100, 'svr__gamma': 'scale', 'svr__kernel': 'linear'}
Mean Squared Error: 5200877604.880186
R-squared: 0.603110350185339
